## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Joao Da Barra,BR,2022-04-11 15:45:16,-21.6403,-41.0511,84.29,67,10,9.33,clear sky
1,1,Saint Augustine,US,2022-04-11 15:49:25,29.8947,-81.3145,77.11,38,0,6.91,clear sky
2,2,Mahon,ES,2022-04-11 15:49:27,39.8885,4.2658,61.36,77,20,21.85,few clouds
3,3,Pisco,PE,2022-04-11 15:44:44,-13.7000,-76.2167,73.45,60,0,14.97,clear sky
4,4,Hobart,AU,2022-04-11 15:47:38,-42.8794,147.3294,51.06,75,75,12.66,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Joao Da Barra,BR,2022-04-11 15:45:16,-21.6403,-41.0511,84.29,67,10,9.33,clear sky
1,1,Saint Augustine,US,2022-04-11 15:49:25,29.8947,-81.3145,77.11,38,0,6.91,clear sky
9,9,Saint Pete Beach,US,2022-04-11 15:49:28,27.7253,-82.7412,79.09,55,0,8.05,clear sky
14,14,Tautira,PF,2022-04-11 15:49:30,-17.7333,-149.1500,80.04,77,24,13.15,light rain
17,17,Lorengau,PG,2022-04-11 15:49:31,-2.0226,147.2712,82.11,77,14,10.80,few clouds
18,18,Poum,NC,2022-04-11 15:49:31,-20.2333,164.0167,78.78,76,8,9.46,clear sky
19,19,Anloga,GH,2022-04-11 15:49:32,5.7947,0.8973,85.39,76,85,14.27,overcast clouds
27,27,Muncar,ID,2022-04-11 15:49:35,-8.4333,114.3333,75.54,82,84,0.69,broken clouds
29,29,Avarua,CK,2022-04-11 15:49:35,-21.2078,-159.7750,75.25,100,20,3.44,few clouds
30,30,Vanimo,PG,2022-04-11 15:49:35,-2.6741,141.3028,78.28,90,23,7.83,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                223
City                   223
Country                223
Date                   223
Lat                    223
Lng                    223
Max Temp               223
Humidity               223
Cloudiness             223
Wind Speed             223
Current Description    223
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sao Joao Da Barra,BR,84.29,clear sky,-21.6403,-41.0511,
1,Saint Augustine,US,77.11,clear sky,29.8947,-81.3145,
9,Saint Pete Beach,US,79.09,clear sky,27.7253,-82.7412,
14,Tautira,PF,80.04,light rain,-17.7333,-149.1500,
17,Lorengau,PG,82.11,few clouds,-2.0226,147.2712,
18,Poum,NC,78.78,clear sky,-20.2333,164.0167,
19,Anloga,GH,85.39,overcast clouds,5.7947,0.8973,
27,Muncar,ID,75.54,broken clouds,-8.4333,114.3333,
29,Avarua,CK,75.25,few clouds,-21.2078,-159.7750,
30,Vanimo,PG,78.28,few clouds,-2.6741,141.3028,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))